## Trade Simulator as a solution to Set Partitioning

In [1]:
import requests
import pandas as pd
import numpy as np
from dimod import BinaryQuadraticModel
from dwave.system import LeapHybridSampler
from random import randrange, sample
from dimod import ExactSolver

In [2]:
url = 'https://fantasy.premierleague.com/api/bootstrap-static/'
r = requests.get(url)
json = r.json()
json.keys()

elements_df = pd.DataFrame(json['elements'])
elements_types_df = pd.DataFrame(json['element_types'])
teams_df = pd.DataFrame(json['teams'])


slim_elements_df = elements_df[['second_name','team','element_type','selected_by_percent','now_cost','minutes','transfers_in','value_season','total_points']]
df = slim_elements_df.merge(elements_types_df, left_on = 'element_type', right_on = 'id')

df = df[['plural_name_short','second_name','team','element_type','selected_by_percent','now_cost','minutes','transfers_in','value_season','total_points']]

df = df.merge(teams_df, left_on = 'team', right_on = 'id')

df = df[['name', 'plural_name_short','second_name','team','selected_by_percent','now_cost','minutes','transfers_in','value_season','total_points']]

df = df[df.minutes > 100]
df.loc[df.plural_name_short == 'GKP', 'class'] = 1
df.loc[df.plural_name_short == 'DEF', 'class'] = 2
df.loc[df.plural_name_short == 'FWD', 'class'] = 3
df.loc[df.plural_name_short == 'MID', 'class'] = 4

arsenal_players_df = df[df.team == 1] # get all the aresenal players
arsenal_players_df.head(1)

df = df[df.team != 1]
tm_dict = dict(tuple(df.groupby('team')))
df.head()

,name,plural_name_short,second_name,team,selected_by_percent,now_cost,minutes,transfers_in,value_season,total_points,class
30,Aston Villa,GKP,Martínez,2,39.7,55,3420,0,33.8,186,1.0
32,Aston Villa,MID,Traoré,2,2.0,60,2315,0,22.5,135,4.0
33,Aston Villa,MID,Grealish,2,35.1,80,2183,0,16.9,135,4.0
34,Aston Villa,MID,Sanson,2,0.1,50,288,0,2.0,10,4.0
35,Aston Villa,MID,McGinn,2,0.8,60,3330,0,19.0,114,4.0


In this notebook we run a trade simulator 100 times to see possible interations between our favourite team (Arsenal) and other teams in the Premier league. We use DWave's exact solver to find the solution for the set partitioning between (for example) 2 players on each team changing sides in some way (one player may change teams or both depending on the solution). We added a custom built in constraint that requires Arsenal to come out ahead at least 1% in any trade, and doesn't allow the opposing team to lose more than 10% on the deal. 

In [6]:
for i in range(100):
    trade_list1 = arsenal_players_df.iloc[sample(range(0, len(arsenal_players_df)), 2)][['plural_name_short', 'value_season', 'second_name']].values.tolist()
    
    random_num = randrange(2,18)
    if random_num != 3:
        try:
            trade_list2 =  tm_dict[random_num].iloc[sample(range(0, len(tm_dict[random_num])), 2)][['plural_name_short', 'value_season', 'second_name']].values.tolist()
        except:
            pass
    
    ####constraint on players allowed in trade###
    
    if (sum([float(trade_list2[0][1]), float(trade_list2[1][1])]) >= 1.01*sum([float(trade_list1[0][1]), float(trade_list1[1][1])])) & \
                                    (sum([float(trade_list2[0][1]), float(trade_list2[1][1])]) < 1.1*sum([float(trade_list1[0][1]), float(trade_list1[1][1])])):

        values = [float(trade_list2[0][1]), float(trade_list2[1][1])] + [float(trade_list1[0][1]), float(trade_list1[1][1])]
        names = [trade_list2[0][2], trade_list2[1][2]] + [trade_list1[0][2], trade_list1[1][2]]
        positions = [trade_list2[0][0], trade_list2[1][0]] + [trade_list1[0][0], trade_list1[1][0]]
        
        bqm = BinaryQuadraticModel('BINARY')
        n = len(values)
        x = {i: bqm.add_variable(f'x_{i}') for i in range(n)}

        bqm.add_linear_equality_constraint(
            [(x[i], 2.0 * values[i]) for i in range(n)],
            constant=-sum(values),
            lagrange_multiplier=10
        )
        
        response = ExactSolver().sample(bqm).truncate(5)
        solution = response.first.sample
        
        set1 = {values[i] for i in x if solution[x[i]]}
        set2 = {values[i] for i in x if not solution[x[i]]}
        
        set1_names = {(names[i], positions[i]) for i in x if solution[x[i]]}
        set2_names = {(names[i], positions[i]) for i in x if not solution[x[i]]}
        
        print('Breaking News! A Trade occured!')
        print(' ')
        print('Before')
        print('Players on Arsenal: ' + str(trade_list1))
        print('Players on Team ' + str(tm_dict[random_num]['name'].iloc[0]) + ' ' + str(trade_list2))
        print(' ')
        print('After')
        
        
        print(f'Sum of value: {sum(set1)}; players now on Arsenal {tuple(set1_names)}')
        print(f'Sum of value: {sum(set2)}; players on ' + str(tm_dict[random_num]['name'].iloc[0]) + ' '  +  f'{tuple(set2_names)}')
        print(' ')
        
    else:
        pass

Breaking News! A Trade occured!
 
Before
Players on Arsenal: [['FWD', '15.2', 'Lacazette'], ['DEF', '8.0', 'Chambers']]
Players on Team Everton [['MID', '10.4', 'Marques Loureiro'], ['MID', '14.5', 'Doucouré']]
 
After
Sum of value: 24.9; players now on Arsenal (('Marques Loureiro', 'MID'), ('Doucouré', 'MID'))
Sum of value: 23.2; players on Everton (('Chambers', 'DEF'), ('Lacazette', 'FWD'))
 
Breaking News! A Trade occured!
 
Before
Players on Arsenal: [['DEF', '7.6', 'Marí'], ['FWD', '15.2', 'Lacazette']]
Players on Team Newcastle [['DEF', '10.4', 'Ritchie'], ['MID', '13.8', 'Murphy']]
 
After
Sum of value: 24.200000000000003; players now on Arsenal (('Ritchie', 'DEF'), ('Murphy', 'MID'))
Sum of value: 22.799999999999997; players on Newcastle (('Marí', 'DEF'), ('Lacazette', 'FWD'))
 
Breaking News! A Trade occured!
 
Before
Players on Arsenal: [['FWD', '13.1', 'Aubameyang'], ['DEF', '15.6', 'Magalhães']]
Players on Team Leicester [['DEF', '20.2', 'Justin'], ['MID', '9.7', 'Pérez']]


KeyError: 3